   O seguinte script adiciona a informação do código do município (IBGE) na base de dados dos cardápios, extrai informações do censo escolar por cidade e por ano e a partir daí junta as informações dos cardápios com as do censo escolar de cada ano por meio do código do município.

In [ ]:
#Importação das bibliotecas
import pandas as pd
from unidecode import unidecode

In [ ]:
#Importação do painel de dados com os cardápios 
df_painel = pd.read_csv(r'C:\Users\guilh\Documents\Pesquisa\painel_cardapios_com_classificacao_alimentos.csv')

In [ ]:
#Importação dos dados com os códigos IBGE dos municípios, disponível em: 'https://www.ibge.gov.br/explica/codigos-dos-municipios.php'
df_mun = pd.read_csv(r'C:\Users\guilh\Documents\Pesquisa\RELATORIO_DTB_BRASIL_MUNICIPIO.csv')

In [ ]:
#Filtro para considerarmos apenas municípios do estado de SP
df_mun = df_mun[df_mun['UF'] == 35]

In [ ]:
#Filtro para considerarmos apenas as colunas com as informações do nome do município e do seu código
df_mun = df_mun[['Código Município Completo','Nome_Município']]

In [ ]:
#Manipulação na forma da escrita dos municípios para evitar possíveis erros por falta de acento, espaço, ponto etc.
df_mun['cid'] = df_mun['Nome_Município'].str.lower().str.replace(" ","").str.replace(",","").str.replace(".","").str.replace("-","").str.replace(":","").apply(lambda s:unidecode(s) if type(s) == str else s)
df_painel['cid'] = df_painel['Nome_cidade'].str.lower().str.replace(" ","").str.replace(",","").str.replace(".","").str.replace("-","").str.replace(":","").apply(lambda s:unidecode(s) if type(s) == str else s)

In [ ]:
#Junção dos dados dos cardápios com a base dos municípios
df_painel = pd.merge(df_painel,df_mun,how='left',on=['cid']) #Diferença no NA por causa de 'escolas' e 'anhanguera'

In [ ]:
df_painel.drop(columns=['cid','Nome_Município'],inplace=True)

## Censo Escolar

   Para explicar a montagem da base de dados com as informações dos censos escolares farei uma explicação separada para um ano específico(2015). Para os demais anos farei de uma forma operacionalizada (por meio de estruturas de controle).
   Fonte dos dados: download feito em https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/censo-escolar para os anos de 2015 a 2019.

Obtendo o número de escolas por cidade

In [ ]:
#Importação dos dados ESCOLAS
df = pd.read_csv(r'C:\Users\guilh\Documents\CENSO_ESCOLAR\2015\DADOS\ESCOLAS.csv',sep='|',encoding='Latin1',low_memory=False,
                usecols=['CO_ENTIDADE','CO_UF','CO_MUNICIPIO','TP_DEPENDENCIA','TP_SITUACAO_FUNCIONAMENTO'])

In [ ]:
#Selecionando as escolas com UF = 35 (Estado de SP), TP_DEPENDENCIA = 2 (Escolas Estaduais) e TP_SITUACAO_FUNCIONAMENTO = 1(Escolas 'Em Atividade') 
df_sp = df[(df['CO_UF'] == 35)&(df['TP_DEPENDENCIA'] == 2)&(df['TP_SITUACAO_FUNCIONAMENTO'] == 1)]

In [ ]:
#Agrupando por código de município, fazendo a contagem e levando adiante apenas a coluna com o código do município e a contagem de CO_ENTIDADE, i.e, número de escolas daquela localidade 
df_sp = df_sp.groupby('CO_MUNICIPIO').count().reset_index()[['CO_MUNICIPIO','CO_ENTIDADE']]

Obtendo o número de estudantes

In [ ]:
#Importação dos dados MATRICULA_SUDESTE
chunk = pd.read_csv(r'C:\Users\guilh\Documents\CENSO_ESCOLAR\2015\DADOS\MATRICULA_SUDESTE.csv',sep='|',encoding='Latin1',low_memory=False,
                    usecols=['CO_PESSOA_FISICA','NU_IDADE','CO_UF','CO_MUNICIPIO','TP_DEPENDENCIA'],chunksize = 100000)

In [ ]:
#Selecionando as escolas com UF = 35 (Estado de SP) e TP_DEPENDENCIA = 2 (Escolas Estaduais) obs: para receber alunos a escola deve estar funcionando, logo não existe o filtro TP_SITUACAO_FUNCIONAMENTO = 1(Escolas 'Em Atividade')  
ls = []
for data in chunk:
    data = data[(data['CO_UF'] == 35)&(data['TP_DEPENDENCIA'] == 2)]
    ls.append(data)
df_estud = pd.concat(ls)

In [ ]:
#Retirando duplicatas na coluna ID dos matriculados, para considerarmos cada aluno apenas uma vez 
df_estud = df_estud.drop_duplicates(subset=['CO_PESSOA_FISICA'])

In [ ]:
#Agrupando por código de município, fazendo a contagem e levando adiante apenas a coluna com o código do município e a contagem de diferentes CO_PESSOA_FISICA, i.e, número de alunos
df_estud = df_estud.groupby(['CO_MUNICIPIO']).count().reset_index()[['CO_MUNICIPIO','CO_PESSOA_FISICA']]

Junção dos dados

In [ ]:
#Reunindo as informações obtidas do número de escola e do número de alunos por município
pd.merge(df_sp,df_estud,on=['CO_MUNICIPIO'])

Operacionalização do processo e restante do código

In [ ]:
#Cabe observar que 2018 e 2019 'caem' em outro loop por causa da alteração do nome da coluna que faz referência ao ID do aluno, antes era 'CO_PESSOA_FISICA' e a partir de 2018 virou 'ID_ALUNO'
dfs = []
for i in range(2015,2020):
    if (i==2015)|(i==2016)|(i==2017):
        df = pd.read_csv(rf'C:\Users\guilh\Documents\CENSO_ESCOLAR\{i}\DADOS\ESCOLAS.csv',sep='|',encoding='Latin1',low_memory=False,
                    usecols=['CO_ENTIDADE','CO_UF','CO_MUNICIPIO','TP_DEPENDENCIA','TP_SITUACAO_FUNCIONAMENTO'])
        df_sp = df[(df['CO_UF'] == 35)&(df['TP_DEPENDENCIA'] == 2)&(df['TP_SITUACAO_FUNCIONAMENTO'] == 1)]
        df_sp = df_sp.groupby('CO_MUNICIPIO').count().reset_index()[['CO_MUNICIPIO','CO_ENTIDADE']]
        chunk = pd.read_csv(rf'C:\Users\guilh\Documents\CENSO_ESCOLAR\{i}\DADOS\MATRICULA_SUDESTE.csv',sep='|',encoding='Latin1',low_memory=False,
                        usecols=['CO_PESSOA_FISICA','NU_IDADE','CO_UF','CO_MUNICIPIO','TP_DEPENDENCIA'],chunksize = 100000)
        ls = []
        for data in chunk:
            data = data[(data['CO_UF'] == 35)&(data['TP_DEPENDENCIA'] == 2)]
            ls.append(data)
        df_estud = pd.concat(ls)
        df_estud = df_estud.drop_duplicates(subset=['CO_PESSOA_FISICA'])
        df_estud = df_estud.groupby(['CO_MUNICIPIO']).count().reset_index()[['CO_MUNICIPIO','CO_PESSOA_FISICA']]
        df = pd.merge(df_sp,df_estud,on=['CO_MUNICIPIO'])
        df['ano'] = i
        print(i)
        dfs.append(df)
    else:
        df = pd.read_csv(rf'C:\Users\guilh\Documents\CENSO_ESCOLAR\{i}\DADOS\ESCOLAS.csv',sep='|',encoding='Latin1',low_memory=False,
                    usecols=['CO_ENTIDADE','CO_UF','CO_MUNICIPIO','TP_DEPENDENCIA','TP_SITUACAO_FUNCIONAMENTO'])
        df_sp = df[(df['CO_UF'] == 35)&(df['TP_DEPENDENCIA'] == 2)&(df['TP_SITUACAO_FUNCIONAMENTO'] == 1)]
        df_sp = df_sp.groupby('CO_MUNICIPIO').count().reset_index()[['CO_MUNICIPIO','CO_ENTIDADE']]
        chunk = pd.read_csv(rf'C:\Users\guilh\Documents\CENSO_ESCOLAR\{i}\DADOS\MATRICULA_SUDESTE.csv',sep='|',encoding='Latin1',low_memory=False,
                        usecols=['ID_ALUNO','NU_IDADE','CO_UF','CO_MUNICIPIO','TP_DEPENDENCIA'],chunksize = 100000)
        ls = []
        for data in chunk:
            data = data[(data['CO_UF'] == 35)&(data['TP_DEPENDENCIA'] == 2)]
            ls.append(data)
        df_estud = pd.concat(ls)
        df_estud = df_estud.drop_duplicates(subset=['ID_ALUNO'])
        df_estud = df_estud.groupby(['CO_MUNICIPIO']).count().reset_index()[['CO_MUNICIPIO','ID_ALUNO']]
        df = pd.merge(df_sp,df_estud,on=['CO_MUNICIPIO'])
        df['ano'] = i
        print(i)
        dfs.append(df)

In [ ]:
#Concatenação em um único dataframe
df_censo = pd.concat(dfs)

In [ ]:
#Teste para averiguarmos se não há inconsistência
df_censo[(df_censo['CO_PESSOA_FISICA'].isna())&(df_censo['ID_ALUNO'].isna())]

In [ ]:
#Substituindo valores NA por 0
df_censo['CO_PESSOA_FISICA'].fillna(0,inplace=True)
df_censo['ID_ALUNO'].fillna(0,inplace=True)

In [ ]:
#Agrupando as informações do número de alunos dos diferentes anos em uma mesma coluna
df_censo['n_alunos'] = df_censo['CO_PESSOA_FISICA'] + df_censo['ID_ALUNO']

In [ ]:
df_censo.drop(columns=['CO_PESSOA_FISICA','ID_ALUNO'],inplace=True)

In [ ]:
#Junção das informações dos cardápios com as do censo escolar por meio do código do município e do ano
df_vf = pd.merge(df_painel,df_censo,how='left',left_on=['Código Município Completo','ano'],right_on=['CO_MUNICIPIO','ano'])

In [ ]:
df_vf.head(70)

In [ ]:
df_vf.drop(columns=['CO_MUNICIPIO'],inplace=True)

In [ ]:
df_vf.set_index(['Nome_cidade','ano']).to_excel(r'painel_cardapios_com_censo_escolar.xlsx')